__Kaggle competition - house prices__

1. [Import](#Import)
    1. [Tools](#Tools)
    1. [Data](#Data)    
1. [EDA](#EDA)
    1. [Category feature EDA](#Category-feature-EDA)
    1. [Count feature EDA](#Count-feature-EDA)
    1. [Continuous feature EDA](#Continuous-feature-EDA)
    1. [Faceting](#Faceting)
    1. [Target variable evaluation](#Target-variable-evaluation)    
1. [Data preparation](#Data-preparation)
    1. [Missing data](#Missing-data)
    1. [Feature engineering](#Feature-engineering)
        1. [Handcrafted](#Handcrafted)
        1. [Polynomial features](#Polynomial-features)
        1. [Encoding](#Encoding)
    1. [Feature transformation](#Feature-transformation)
        1. [Skew correction](#Skew-correction)
        1. [Scaling](#Scaling)     
    1. [Outliers](#Outliers)
    1. [Additional exploratory data analysis](#Additional-exploratory-data-analysis)
1. [Feature importance](#Feature-importance)    
1. [Modeling](#Modeling)
    1. [Data preparation](#Data-preparation-1)
    1. [Bayesian hyper-parameter optimization](#Bayesian-hyper-parameter-optimization)
    1. [Model performance evaluation - standard models](#Model-performance-evaluation-standard-models)
    1. [Model explanability](#Model-explanability)
    1. [Submission - standard models](#Submission-standard-models)
1. [Stacking](#Stacking)
    1. [Primary models](#Primary-models)
    1. [Meta model](#Meta-model)                
    1. [Model performance evaluation - stacked models](#Model-performance-evaluation-stacked-models)
    1. [Submission - stacked models](#Submission-stacked-models)    

# Import

<a id = 'Import'></a>

## Tools

<a id = 'Tools'></a>

In [ ]:
# standard libary and settings
import copy
import os
import sys
import importlib
import itertools
from functools import reduce
import time

rundate = time.strftime("%Y%m%d")

import warnings

warnings.simplefilter("ignore")

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

# data extensions and settings
import numpy as np

np.set_printoptions(threshold=np.inf, suppress=True)

import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.options.display.float_format = "{:,.6f}".format

# modeling extensions
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.datasets import (
    load_boston,
    load_wine,
    load_iris,
    load_breast_cancer,
    make_blobs,
    make_moons,
)
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
    IsolationForest,
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import (
    Lasso,
    Ridge,
    ElasticNet,
    LinearRegression,
    LogisticRegression,
    SGDRegressor,
)
from sklearn.model_selection import (
    KFold,
    train_test_split,
    GridSearchCV,
    StratifiedKFold,
    cross_val_score,
    RandomizedSearchCV,
)
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    PolynomialFeatures,
    OrdinalEncoder,
    LabelEncoder,
    OneHotEncoder,
    KBinsDiscretizer,
    QuantileTransformer,
    PowerTransformer,
    MinMaxScaler,
)
from sklearn.svm import SVC, SVR
from category_encoders import (
    WOEEncoder,
    TargetEncoder,
    CatBoostEncoder,
    BinaryEncoder,
    CountEncoder,
)

from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor

from hyperopt import hp

import eif
import shap

shap.initjs()

# visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import squarify

%matplotlib inline

try:
    #     import mlmachine as mlm
    #     from prettierplot.plotter import PrettierPlot
    #     import prettierplot.style as style
    import asdfasd
except ModuleNotFoundError:
    sys.path.append(
        "../../../mlmachine"
    ) if "../../../../mlmachine" not in sys.path else None
    sys.path.append(
        "../../../prettierplot"
    ) if "../../../../prettierplot" not in sys.path else None

    import mlmachine as mlm
    import mlmachine.data as data
    from mlmachine.features.preprocessing import (
        DataFrameSelector,
        PandasPipeline,
        KFoldSelectEncoder,
        ContextImputer,
        PandasFeatureUnion,
        DualTransformer,
    )
    from prettierplot.plotter import PrettierPlot
    import prettierplot.style as style
else:
    print(
        "This notebook relies on the libraries mlmachine and prettierplot. Please run:"
    )
    print("\tpip install mlmachine")
    print("\tpip install prettierplot")

## Data

<a id = 'Data'></a>

In [ ]:
# load data and print dimensions
df_train, df_valid = data.housing()

print("Training data dimensions: {}".format(df_train.shape))
print("Validation data dimensions: {}".format(df_valid.shape))

In [ ]:
# display info and first 5 rows
df_train.info()
display(df_train[:5])

In [ ]:
# review counts of different column types
df_train.dtypes.value_counts()

In [ ]:
continuous = [
    "LotFrontage",
    "LotArea",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "LowQualFinSF",
    "GrLivArea",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch",
    "PoolArea",
]

remove_features = [
    "Id",
    "MiscVal",
]

count = [
    "BsmtFullBath",
    "BsmtHalfBath",
    "FullBath",
    "HalfBath",
    "BedroomAbvGr",
    "KitchenAbvGr",
    "TotRmsAbvGrd",
    "Fireplaces",
    "GarageCars",
]

nominal = [
    "MSSubClass",
    "MSZoning",    
    "LandContour",
    "Neighborhood",
    "Condition1",
    "Condition2",
    "BldgType",
    "HouseStyle",
    "YearBuilt",
    "YearRemodAdd",
    "RoofStyle",
    "RoofMatl",
    "Exterior1st",
    "Exterior2nd",
    "MasVnrType",
    "Foundation",
    "Heating",
    "GarageType",
    "GarageYrBlt", 
    "Fence",
    "SaleType",
    "SaleCondition",
    "MiscFeature",    
    "MoSold",
    "YrSold",
    
]

ordinal = [
    "Street",  
    "Alley" ,
    "LotShape", 
    "Utilities", 
    "LotConfig",
    "LandSlope",
    "OverallQual",
    "OverallCond",    
    "ExterQual", 
    "ExterCond", 
    "BsmtQual", 
    "BsmtCond", 
    "BsmtExposure", 
    "BsmtFinType1", 
    "BsmtFinType2", 
    "HeatingQC", 
    "CentralAir", 
    "Electrical", 
    "KitchenQual", 
    "Functional", 
    "FireplaceQu", 
    "GarageFinish", 
    "GarageQual", 
    "GarageCond", 
    "PavedDrive", 
    "PoolQC",    
]

ordinal_encodings = {
    "Street": ["Grvl", "Pave"],
    "Alley": ["Nonexistent", "Grvl", "Pave"],
    "LotShape": ["IR3", "IR2", "IR1", "Reg"],
    "Utilities": ["ELO", "NoSeWa", "NoSewr", "AllPub"],
    "LotConfig": ["FR3", "FR2", "Corner", "Inside", "CulDSac"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "OverallQual": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "OverallCond": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "ExterQual": ["Po", "Fa", "TA", "Gd", "Ex"], 
    "ExterCond": ["Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtQual": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtCond": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtExposure": ["Nonexistent", "No", "Mn", "Av", "Gd"],
    "BsmtFinType1": ["Nonexistent", "Unf", "LwQ", "BLQ", "Rec", "ALQ", "GLQ"],
    "BsmtFinType2": ["Nonexistent", "Unf", "LwQ", "BLQ", "Rec", "ALQ", "GLQ"],
    "HeatingQC": ["Po", "Fa", "TA", "Gd", "Ex"],
    "CentralAir": ["N", "Y"],
    "Electrical": ["FuseP", "FuseF", "FuseA", "Mix", "SBrkr"],
    "KitchenQual": ["Po", "Fa", "TA", "Gd", "Ex"],
    "Functional": ["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"],
    "FireplaceQu": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "GarageFinish": ["Nonexistent", "Unf", "RFn", "Fin"],
    "GarageQual": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "GarageCond": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "PavedDrive": ["N", "P", "Y"],
    "PoolQC": ["Nonexistent", "Fa", "TA", "Gd", "Ex"],
}

In [ ]:
# load training data into mlmachine
train = mlm.Machine(
    data=df_train,
    target="SalePrice",
    remove_features=remove_features,
    identify_as_continuous=continuous,
    identify_as_count=count,    
    identify_as_nominal=nominal,
    identify_as_ordinal=ordinal,
    ordinal_encodings=ordinal_encodings,
    target_type="number",
)

In [ ]:
# load training data into mlmachine
valid = mlm.Machine(
    data=df_valid,
    remove_features=remove_features,
    identify_as_continuous=continuous,
    identify_as_count=count,    
    identify_as_nominal=nominal,
    identify_as_ordinal=ordinal,
    ordinal_encodings=ordinal_encodings,
)

# EDA

<a id = 'EDA'></a>

## Category feature EDA

<a id = 'Category-feature-EDA'></a>

In [ ]:
# number features
for feature in train.data.mlm_dtypes["category"][:1]:
    train.eda_num_target_cat_feat(
        feature=feature,
        level_count_cap=10,
        chart_scale=20
    )

## Count feature EDA

<a id = 'Count-feature-EDA'></a>

In [ ]:
# number features
for feature in train.data.mlm_dtypes["count"][:1]:
    train.eda_num_target_cat_feat(
        feature=feature,
#         level_count_cap=20,
        chart_scale=20
    )

## Continuous feature EDA

<a id = 'Continuous-feature-EDA'></a>

In [ ]:
# continuous features
for feature in train.data.mlm_dtypes["continuous"][:3]:
    train.eda_num_target_num_feat(
        feature=feature,
#         outliers_out_of_scope=5,
        chart_scale=20
    )

##### Correlation

In [ ]:
# correlation heat map
p = PrettierPlot(chart_scale=25)
ax = p.make_canvas()
p.corr_heatmap(df=train.data, ax=ax)

In [ ]:
# correlation heat map with most highly correlated features relative to the target
p = PrettierPlot(plot_orientation='tall',chart_scale=15)
ax = p.make_canvas()
p.corr_heatmap_target(df=train.data, target=train.target, thresh=0.6, annot = True, ax=ax)

> Remarks - There are three pairs of highly correlated features:
    - 'GarageArea' and 'GarageCars'
    - 'TotRmsAbvGrd' and 'GrLivArea'
    - '1stFlrSF' and 'TotalBsmtSF
This makes sense, given what each feature represents and how each pair items relate to each other. We likely only need one feature from each pair.

##### Pair plot

In [ ]:
# pair plot
p = PrettierPlot(chart_scale=10)
p.pair_plot(
    df=train.data,
    columns=[
        "LotFrontage",
        "LotArea",
        "MasVnrArea",
        "BsmtFinSF1",
        "BsmtFinSF2",
        "BsmtUnfSF",
        "TotalBsmtSF",
        "1stFlrSF",
        "2ndFlrSF",
        "GrLivArea",
        "TotRmsAbvGrd",
        "GarageYrBlt",
        "GarageArea",
        "WoodDeckSF",
        "OpenPorchSF",
    ],
    diag_kind="auto",
)

## Faceting

<a id = 'Faceting'></a>

## Target variable evaluation

<a id = 'Target-variable-evaluation'></a>

In [ ]:
# evaluate distribution of target variable
train.eda_transform_initial(data=train.target, name=train.target.name)
train.eda_transform_log1(data=train.target, name=train.target.name)

In [ ]:
# log + 1 transform target
train.target = np.log1p(train.target)

# Data preparation

<a id = 'Data-preparation'></a>

## Missing data


<a id = 'Missing-data'></a>

##### Training

In [ ]:
# evaluate missing data
train.eda_missing_summary(chart_scale=25)

In [ ]:
# missingno matrix
msno.matrix(train.data)

In [ ]:
# missingno bar
msno.bar(train.data)

In [ ]:
# missingno heatmap
msno.heatmap(train.data)

In [ ]:
# missingno dendrogram
msno.dendrogram(train.data)

##### Validation

In [ ]:
# evaluate missing data
valid.eda_missing_summary()

In [ ]:
# missingno matrix
msno.matrix(valid.data)

In [ ]:
# missingno bar
msno.bar(valid.data)

In [ ]:
# missingno heatmap
msno.heatmap(valid.data)

In [ ]:
# missingno dendrogram
msno.dendrogram(valid.data)

##### Training vs. validation


In [ ]:
# compare feature with missing data
train.missing_col_compare(train=train.data, validation=valid.data)

##### Impute


In [ ]:
# impute pipeline
category_constant = ['GarageFinish', 'Alley', 'MasVnrType', 'GarageType', 'BsmtFinType1',
                       'BsmtCond', 'BsmtFinType2', 'BsmtQual', 'PoolQC', 'GarageCond',
                       'FireplaceQu', 'GarageQual', 'Fence', 'BsmtExposure', 'MiscFeature']
number_constant = ["GarageYrBlt","MasVnrArea","BsmtUnfSF","GarageArea","BsmtFinSF1","TotalBsmtSF","BsmtFinSF2"]
category_mode = ["Electrical","Functional","SaleType","Exterior1st","MSZoning","Exterior2nd","KitchenQual","Utilities"]
number_mode = ["BsmtHalfBath", "GarageCars", "BsmtFullBath"]

impute_pipe = PandasFeatureUnion([
    ("catConstant", make_pipeline(
        DataFrameSelector(include_columns=category_constant),
        PandasPipeline(SimpleImputer(strategy="constant", fill_value="Nonexistent"))
    )),
    ("numConstant", make_pipeline(
        DataFrameSelector(include_columns=number_constant),
        PandasPipeline(SimpleImputer(strategy="constant", fill_value=0))
    )),
    ("catMode", make_pipeline(
        DataFrameSelector(include_columns=category_mode),
        PandasPipeline(SimpleImputer(strategy="most_frequent"))
    )),
    ("numMode", make_pipeline(
        DataFrameSelector(include_columns=number_mode),
        PandasPipeline(SimpleImputer(strategy="most_frequent"))
    )),
    ("LotFrontage", make_pipeline(
        DataFrameSelector(include_columns=["LotFrontage","Neighborhood"]),
        ContextImputer(null_col="LotFrontage", context_col="Neighborhood", strategy="mean")
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns=["LotFrontage"] + category_constant + number_constant + category_mode + number_mode),
    )),
])

train.data = impute_pipe.fit_transform(train.data)
valid.data = impute_pipe.transform(valid.data)

In [ ]:
train.eda_missing_summary()

In [ ]:
valid.eda_missing_summary()

## Feature engineering

<a id = 'Feature-engineering'></a>

### Handcrafted

<a id = 'Handcrafted'></a>

In [ ]:
# additional features
train.data["BsmtFinSF"] = train.data["BsmtFinSF1"] + train.data["BsmtFinSF2"]
train.data["TotalSF"] = (
    train.data["TotalBsmtSF"] + train.data["1stFlrSF"] + train.data["2ndFlrSF"]
)
train.update_dtypes()

In [ ]:
# evaluate additional features
for feature in ["BsmtFinSF","TotalSF"]:
    train.eda_num_target_num_feat(
        feature=feature,
#         outliers_out_of_scope=5,
        chart_scale=20
    )

In [ ]:
# additional features
valid.data["BsmtFinSF"] = valid.data["BsmtFinSF1"] + valid.data["BsmtFinSF2"]
valid.data["TotalSF"] = (
    valid.data["TotalBsmtSF"] + valid.data["1stFlrSF"] + valid.data["2ndFlrSF"]
)
valid.update_dtypes()

### Polynomial features

<a id = 'Polynomial-features'></a>

In [ ]:
# transform pipe
polynomial_pipe = PandasFeatureUnion([
    ("polynomial", make_pipeline(
        DataFrameSelector(include_columns=["TotalSF","BsmtFinSF"]),
        #DataFrameSelector(include_mlm_dtypes=["continuous"]),
        PandasPipeline(PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)),
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns=["TotalSF","BsmtFinSF"]),
        #DataFrameSelector(exclude_mlm_dtypes=["continuous"]),
    )),
])

train.data = polynomial_pipe.fit_transform(train.data)
valid.data = polynomial_pipe.transform(valid.data)

train.update_dtypes()
valid.update_dtypes()

### Encoding

<a id = 'Encoding'></a>

In [ ]:
# counts of unique values in training data object columns
train.data[train.data.mlm_dtypes["category"]].apply(pd.Series.nunique, axis=0)

In [ ]:
# print unique values in each object columns
for col in train.data.mlm_dtypes["category"]:
    print(col, np.unique(train.data[col]))

In [ ]:
# counts of unique values in validation data string columns
valid.data[valid.data.mlm_dtypes["category"]].apply(pd.Series.nunique, axis=0)

In [ ]:
# print unique values in each object columns
for col in valid.data.mlm_dtypes["category"]:
    print(col, np.unique(valid.data[col]))

In [ ]:
# identify values that are present in the training data but not the validation data, and vice versa
for col in train.data.mlm_dtypes["category"]:
    train_values = train.data[col].unique()
    valid_values = valid.data[col].unique()

    train_diff = set(train_values) - set(valid_values)
    valid_diff = set(valid_values) - set(train_values)

    if len(train_diff) > 0 or len(valid_diff) > 0:
        print("\n\n*** " + col)
        print("Value present in training data, not in validation data")
        print(train_diff)
        print("Value present in validation data, not in training data")
        print(valid_diff)

In [ ]:
# encode pipeline
encode_pipe = PandasFeatureUnion([
    ("nominal", make_pipeline(
        DataFrameSelector(include_columns=nominal, exclude_columns=["YearBuilt","GarageYrBlt"]),
        PandasPipeline(OneHotEncoder(drop=None, handle_unknown="ignore")),
    )),
    ("ordinal", make_pipeline(
        DataFrameSelector(include_columns=list(ordinal_encodings.keys())),
        PandasPipeline(OrdinalEncoder(categories=list(ordinal_encodings.values()))),
    )),
    ("bin", make_pipeline(
        DataFrameSelector(include_columns=train.data.mlm_dtypes["continuous"]),
        PandasPipeline(KBinsDiscretizer(encode="ordinal")),
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns=nominal + list(ordinal_encodings.keys())),
    )),
])

train.data = encode_pipe.fit_transform(train.data)
valid.data = encode_pipe.transform(valid.data)

train.update_dtypes()
valid.update_dtypes()

In [ ]:
# target encoding pipe
target_encode_pipe = PandasFeatureUnion([
    ("target", make_pipeline(
        DataFrameSelector(include_mlm_dtypes=["category"]),
        KFoldSelectEncoder(
            target=train.target,
            cv=KFold(n_splits=5, shuffle=False, random_state=0),
            encoder=TargetEncoder,
        ),
    )),
    ("catboost", make_pipeline(
        DataFrameSelector(include_mlm_dtypes=["category"]),
        KFoldSelectEncoder(
            target=train.target,
            cv=KFold(n_splits=5, shuffle=False, random_state=0),
            encoder=CatBoostEncoder,
        ),
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_mlm_dtypes=["category"]),
    )),
])

train.data = target_encode_pipe.fit_transform(train.data)
valid.data = target_encode_pipe.transform(valid.data)

train.update_dtypes()
valid.update_dtypes()

## Feature transformation

<a id = 'Feature-transformation'></a>

### Skew correction


<a id = 'Skew-correction'></a>

In [ ]:
# evaluate skew of number features - validation data
train.skew_summary()

In [ ]:
# evaluate skew of number features - training data
valid.skew_summary()

In [ ]:
# skew correction pipeline
skew_pipe = PandasFeatureUnion([
    ("skew", make_pipeline(
        DataFrameSelector(include_mlm_dtypes=["continuous"]),
        DualTransformer(),
    )),    
    ("diff", make_pipeline(
        DataFrameSelector(exclude_mlm_dtypes=["continuous"]),
    )),
])

# train.data = skew_pipe.fit_transform(train.data)
# valid.data = skew_pipe.transform(valid.data)

# train.update_dtypes()
# valid.update_dtypes()

### Scaling


<a id = 'Scaling'></a>

In [ ]:
#
scale_pipe = PandasFeatureUnion([
    ("scale", make_pipeline(
        DataFrameSelector(),
        PandasPipeline(RobustScaler())
    )),
])

train.data = scale_pipe.fit_transform(train.data)
valid.data = scale_pipe.transform(valid.data)

train.update_dtypes()
valid.update_dtypes()

## Outliers

<a id = 'Outliers'></a>

In [ ]:
# identify outliers using IQR
train_pipe = Pipeline([
    ("outlier",train.OutlierIQR(
                outlier_count=20,
                iqr_step=1.5,
                features=train.data.mlm_dtypes["number"],
                drop_outliers=False,))
    ])
train.data = train_pipe.transform(train.data)

# capture outliers
iqr_outliers = np.array(sorted(train_pipe.named_steps["outlier"].outliers))
print(iqr_outliers)

In [ ]:
# identify outliers using Isolation Forest
clf = IsolationForest(
    behaviour="new", max_samples=train.data.shape[0], random_state=0, contamination=0.01
)
clf.fit(train.data[train.data.columns])
preds = clf.predict(train.data[train.data.columns])

# evaluate index values
mask = np.isin(preds, -1)
if_outliers = np.array(train.data[mask].index)
print(if_outliers)

In [ ]:
# identify outliers using extended isolation forest
train_pipe = Pipeline([
    ("outlier",train.ExtendedIsoForest(
                columns=train.data.mlm_dtypes["number"],
                n_trees=100,
                sample_size=256,
                extension_level=1,
                anomalies_ratio=0.03,
                drop_outliers=False,))
    ])
train.data = train_pipe.transform(train.data)

# capture outliers
eif_outliers = np.array(sorted(train_pipe.named_steps["outlier"].outliers))
print(eif_outliers)

In [ ]:
# identify outliers that are identified in multiple algorithms
outliers = reduce(np.intersect1d, (iqr_outliers, if_outliers, eif_outliers))
# outliers = reduce(np.intersect1d, (if_outliers, eif_outliers))
print(outliers)

In [ ]:
# review outlier identification summary
outlier_summary = train.outlier_summary(iqr_outliers=iqr_outliers,
                             if_outliers=if_outliers,
                             eif_outliers=eif_outliers
                            )
outlier_summary[outlier_summary["count"] >= 3]

In [ ]:
# capture index values of known outliers
knownOutliers = (
    train.data[train.data["LotArea"] > 60000].index.values.tolist()
    + train.data[train.data["LotFrontage"] > 300].index.values.tolist()
    + train.data[train.data["GrLivArea"] > 4000].index.values.tolist()
)
knownOutliers = sorted(set(knownOutliers))
print(knownOutliers)

# index of known outliers and outliers identified with the known outliers removed
outliers = [
    53,
    185,
    197,
    437,
    492,
    762,
    796,
    821,
    847,
    1161,
    1221,
    1318,
    1376,
    249,
    313,
    335,
    451,
    523,
    691,
    706,
    934,
    1182,
    1298,
]
print(outliers)

# remove outlers from predictors and response
train.data = train.data.drop(outliers)
train.target = train.target.drop(index=outliers)

print(train.data.shape)
print(train.target.shape)

## Additional exploratory data analysis

<a id = 'Additional-exploratory-data-analysis'></a>

# Feature importance

<a id = 'Feature-importance'></a>

In [ ]:
# generate feature importance summary
estimators = [
    LGBMRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
    XGBRegressor,
]

fs = train.FeatureSelector(
    data=train.data,
    target=train.target,
    estimators=estimators,
    classification=False,
)
feature_selector_summary = fs.feature_selector_suite(
    save_to_csv=True,
    n_jobs=5,
)


In [ ]:
# calculate cross-validation performance
estimators = [
    SVR,
    LGBMRegressor,
    XGBRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    ExtraTreesRegressor,
    KNeighborsRegressor,
]

cv_summary = fs.feature_selector_cross_val(
    feature_selector_summary=feature_selector_summary,
    estimators=estimators,
    scoring=["root_mean_squared_error"],
    n_folds=5,
    step=1,
    n_jobs=5,
)

###### Mean squared error

In [ ]:
# visualize CV performance for diminishing feature set
fs.feature_selector_results_plot(
    scoring="root_mean_squared_error",
    feature_selector_summary=feature_selector_summary,
    cv_summary=cv_summary,
#     feature_selector_summary=".csv",
#     cv_summary=".csv",
    show_features=False,
    marker_on=False,
    title_scale=0.8,
)

In [ ]:
cross_val_features_df = fs.create_cross_val_features_df(
    metric="root_mean_squared_error",
    cv_summary= pd.read_csv("cv_summary_20191028_125617.csv", index_col=0),
    feature_selector_summary=pd.read_csv("featureSelectionSummary_20191027_022938.csv", index_col=0),
)
cross_val_features_df#[:5]

In [ ]:
cross_val_feature_dict = fs.create_cross_val_features_dict(
    cross_val_features_df=cross_val_features_df
)

# Modeling

<a id = 'Modeling'></a>

## Data preparation

<a id = 'Data-preparation-1'></a>

In [ ]:
#################################################################################
# import training data
df_train, df_valid = data.housing()

continuous = [
    "LotFrontage",
    "LotArea",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "LowQualFinSF",
    "GrLivArea",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch",
    "PoolArea",
]

remove_features = [
    "Id",
    "MiscVal",
]

count = [
    "BsmtFullBath",
    "BsmtHalfBath",
    "FullBath",
    "HalfBath",
    "BedroomAbvGr",
    "KitchenAbvGr",
    "TotRmsAbvGrd",
    "Fireplaces",
    "GarageCars",
]

nominal = [
    "MSSubClass",
    "MSZoning",    
    "LandContour",
    "Neighborhood",
    "Condition1",
    "Condition2",
    "BldgType",
    "HouseStyle",
    "YearBuilt",
    "YearRemodAdd",
    "RoofStyle",
    "RoofMatl",
    "Exterior1st",
    "Exterior2nd",
    "MasVnrType",
    "Foundation",
    "Heating",
    "GarageType",
    "GarageYrBlt", 
    "Fence",
    "SaleType",
    "SaleCondition",
    "MiscFeature",    
    "MoSold",
    "YrSold",
    
]

ordinal = [
    "Street",  
    "Alley" ,
    "LotShape", 
    "Utilities", 
    "LotConfig",
    "LandSlope",
    "OverallQual",
    "OverallCond",    
    "ExterQual", 
    "ExterCond", 
    "BsmtQual", 
    "BsmtCond", 
    "BsmtExposure", 
    "BsmtFinType1", 
    "BsmtFinType2", 
    "HeatingQC", 
    "CentralAir", 
    "Electrical", 
    "KitchenQual", 
    "Functional", 
    "FireplaceQu", 
    "GarageFinish", 
    "GarageQual", 
    "GarageCond", 
    "PavedDrive", 
    "PoolQC",    
]

ordinal_encodings = {
    "Street": ["Grvl", "Pave"],
    "Alley": ["Nonexistent", "Grvl", "Pave"],
    "LotShape": ["IR3", "IR2", "IR1", "Reg"],
    "Utilities": ["ELO", "NoSeWa", "NoSewr", "AllPub"],
    "LotConfig": ["FR3", "FR2", "Corner", "Inside", "CulDSac"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "OverallQual": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "OverallCond": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "ExterQual": ["Po", "Fa", "TA", "Gd", "Ex"], 
    "ExterCond": ["Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtQual": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtCond": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtExposure": ["Nonexistent", "No", "Mn", "Av", "Gd"],
    "BsmtFinType1": ["Nonexistent", "Unf", "LwQ", "BLQ", "Rec", "ALQ", "GLQ"],
    "BsmtFinType2": ["Nonexistent", "Unf", "LwQ", "BLQ", "Rec", "ALQ", "GLQ"],
    "HeatingQC": ["Po", "Fa", "TA", "Gd", "Ex"],
    "CentralAir": ["N", "Y"],
    "Electrical": ["FuseP", "FuseF", "FuseA", "Mix", "SBrkr"],
    "KitchenQual": ["Po", "Fa", "TA", "Gd", "Ex"],
    "Functional": ["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"],
    "FireplaceQu": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "GarageFinish": ["Nonexistent", "Unf", "RFn", "Fin"],
    "GarageQual": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "GarageCond": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "PavedDrive": ["N", "P", "Y"],
    "PoolQC": ["Nonexistent", "Fa", "TA", "Gd", "Ex"],
}

train = mlm.Machine(
    data=df_train,
    target="SalePrice",
    remove_features=remove_features,
    identify_as_continuous=continuous,
    identify_as_count=count,    
    identify_as_nominal=nominal,
    identify_as_ordinal=ordinal,
    ordinal_encodings=ordinal_encodings,
    target_type="number",
)

# additional features
train.data["BsmtFinSF"] = train.data["BsmtFinSF1"] + train.data["BsmtFinSF2"]
train.data["TotalSF"] = (
    train.data["TotalBsmtSF"] + train.data["1stFlrSF"] + train.data["2ndFlrSF"]
)

#################################################################################
# import validation data
valid = mlm.Machine(
    data=df_valid,
    remove_features=remove_features,
    identify_as_continuous=continuous,
    identify_as_count=count,    
    identify_as_nominal=nominal,
    identify_as_ordinal=ordinal,
    ordinal_encodings=ordinal_encodings,
)

# change clearly erroneous value to what it probably was
valid.data["GarageYrBlt"].replace({2207: 2007}, inplace=True)

# additional features
valid.data["BsmtFinSF"] = valid.data["BsmtFinSF1"] + valid.data["BsmtFinSF2"]
valid.data["TotalSF"] = (
    valid.data["TotalBsmtSF"] + valid.data["1stFlrSF"] + valid.data["2ndFlrSF"]
)
valid.data.loc[valid.data["TotalSF"].isnull(), "TotalSF"] = (
    valid.data["1stFlrSF"] + valid.data["2ndFlrSF"]
)

train.update_dtypes()
valid.update_dtypes()

#################################################################################
# impute pipeline
object_constant = ['GarageFinish', 'Alley', 'MasVnrType', 'GarageType', 'BsmtFinType1',
                       'BsmtCond', 'BsmtFinType2', 'BsmtQual', 'PoolQC', 'GarageCond',
                       'FireplaceQu', 'GarageQual', 'Fence', 'BsmtExposure', 'MiscFeature']
number_constant = ["GarageYrBlt","MasVnrArea","BsmtUnfSF","GarageArea","BsmtFinSF","BsmtFinSF1","TotalBsmtSF","BsmtFinSF2"]
object_mode = ["Electrical","Functional","SaleType","Exterior1st","MSZoning","Exterior2nd","KitchenQual","Utilities"]
number_mode = ["BsmtHalfBath", "GarageCars", "BsmtFullBath"]

impute_pipe = PandasFeatureUnion([
    ("catConstant", make_pipeline(
        DataFrameSelector(object_constant),
        PandasPipeline(SimpleImputer(strategy="constant", fill_value="Nonexistent"))
    )),
    ("numConstant", make_pipeline(
        DataFrameSelector(number_constant),
        PandasPipeline(SimpleImputer(strategy="constant", fill_value=0))
    )),
    ("catMode", make_pipeline(
        DataFrameSelector(object_mode),
        PandasPipeline(SimpleImputer(strategy="most_frequent"))
    )),
    ("numMode", make_pipeline(
        DataFrameSelector(number_mode),
        PandasPipeline(SimpleImputer(strategy="most_frequent"))
    )),
    ("LotFrontage", make_pipeline(
        DataFrameSelector(["LotFrontage","Neighborhood"]),
        ContextImputer(null_col="LotFrontage", context_col="Neighborhood", strategy="mean")
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns = ["LotFrontage"] + object_constant + number_constant + object_mode + number_mode),
    )),
])

train.data = impute_pipe.fit_transform(train.data)
valid.data = impute_pipe.transform(valid.data)

#################################################################################
# polynomial feature pipe
polynomial_pipe = PandasFeatureUnion([
    ("polynomial", make_pipeline(
        DataFrameSelector(include_mlm_dtypes=["continuous"]),
        PandasPipeline(PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)),
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_mlm_dtypes=["continuous"], exclude_columns=["Name","Cabin"]),
    )),
])

train.data = polynomial_pipe.fit_transform(train.data)
valid.data = polynomial_pipe.transform(valid.data)

train.update_dtypes()
valid.update_dtypes()

# feature transformation pipeline
encode_pipe = PandasFeatureUnion([
    ("nominal", make_pipeline(
        DataFrameSelector(include_columns=nominal, exclude_columns=["YearBuilt","GarageYrBlt"]),
        PandasPipeline(OneHotEncoder(drop=None, handle_unknown="ignore")),
    )),
    ("ordinal", make_pipeline(
        DataFrameSelector(include_columns=list(ordinal_encodings.keys())),
        PandasPipeline(OrdinalEncoder(categories=list(ordinal_encodings.values()))),
    )),
    ("bin", make_pipeline(
        DataFrameSelector(include_columns=train.data.mlm_dtypes["continuous"]),
        PandasPipeline(KBinsDiscretizer(encode="ordinal")),
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns=nominal + list(ordinal_encodings.keys())),
    )),
])

train.data = encode_pipe.fit_transform(train.data)
valid.data = encode_pipe.transform(valid.data)

train.update_dtypes()
valid.update_dtypes()

#################################################################################
# remove outliers
outliers = [
    53,
    185,
    197,
    437,
    492,
    762,
    796,
    821,
    847,
    1161,
    1221,
    1318,
    1376,
    249,
    313,
    335,
    451,
    523,
    691,
    706,
    934,
    1182,
    1298,
]
train.data = train.data.drop(outliers)
train.target = train.target.drop(index=outliers)

# log transform target
train.target = np.log1p(train.target)
print('completed')

## Bayesian hyper-parameter optimization

<a id = 'Bayesian-hyper-parameter-optimization'></a>

In [ ]:
# model/parameter space
all_space = {
    "Lasso": {"alpha": hp.uniform("alpha", 0.0000001, 20)},
    "Ridge": {"alpha": hp.uniform("alpha", 0.0000001, 20)},
    "ElasticNet": {
        "alpha": hp.uniform("alpha", 0.0000001, 20),
        "l1_ratio": hp.uniform("l1_ratio", 0.0, 0.2),
    },
    "KernelRidge": {
        "alpha": hp.uniform("alpha", 0.000001, 15),
        "kernel": hp.choice("kernel", ["linear", "polynomial", "rbf"]),
        "degree": hp.choice("degree", [2, 3]),
        "gamma": hp.uniform("gamma", 0.0, 10),
    },
    "LGBMRegressor": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "boosting_type": hp.choice("boosting_type", ["gbdt", "dart", "goss"])
        # ,'boosting_type': hp.choice('boosting_type'
        #                    ,[{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}
        #                    ,{'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)}
        #                    ,{'boosting_type': 'goss', 'subsample': 1.0}])
        ,
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_child_samples": hp.uniform("min_child_samples", 20, 500),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "num_leaves": hp.uniform("num_leaves", 8, 150),
        "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "subsample_for_bin": hp.uniform("subsample_for_bin", 20000, 400000),
    },
    "XGBRegressor": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "gamma": hp.uniform("gamma", 0.0, 10),
        "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_child_weight": hp.uniform("min_child_weight", 1, 20),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "subsample": hp.uniform("subsample", 0.5, 1),
    },
    "RandomForestRegressor": {
        "bootstrap": hp.choice("bootstrap", [True, False]),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "GradientBoostingRegressor": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "loss": hp.choice("loss", ["ls", "lad", "huber", "quantile"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "AdaBoostRegressor": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "loss": hp.choice("loss", ["linear", "square", "exponential"]),
    },
    "ExtraTreesRegressor": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
    },
    "SVR": {
        "C": hp.uniform("C", 0.00001, 10),
        "kernel": hp.choice("kernel", ["linear", "poly", "rbf", "sigmoid"]),
        "degree": hp.choice("degree", [2, 3]),
        "gamma": hp.uniform("gamma", 0.0001, 10),
        "epsilon": hp.uniform("epsilon", 0.001, 5),
    },
    "KNeighborsRegressor": {
        "algorithm": hp.choice("algorithm", ["auto", "ball_tree", "kd_tree", "brute"]),
        "n_neighbors": hp.choice("n_neighbors", np.arange(1, 20, dtype=int)),
        "weights": hp.choice("weights", ["distance", "uniform"]),
        "p": hp.choice("p", [1, 2]),
    },
}

In [ ]:
# execute bayesian optimization grid search
train.exec_bayes_optim_search(
    all_space=all_space,
    data=train.data,
    target=train.target,
    scoring="root_mean_squared_error",
    columns=cross_val_feature_dict,
    n_folds=5,
    n_jobs=8,
    iters=750,
    verbose=0,
)

##### Model loss by iteration

In [ ]:
# read scores summary table
bayes_optim_summary = pd.read_csv("", na_values="nan")
bayes_optim_summary[:5]

In [ ]:
# model loss plot
for estimator in np.unique(bayes_optim_summary["estimator"]):
    train.model_loss_plot(bayes_optim_summary=bayes_optim_summary, estimator=estimator)

##### Parameter selection by iteration

In [ ]:
# estimator parameter plots
for estimator in np.unique(bayes_optim_summary['estimator']):
    train.modelParamPlot(bayes_optim_summary = bayes_optim_summary,
                         estimator=estimator,
                         all_space=all_space,
                         n_iter=100,
                         chart_scale=15)

In [ ]:
sample_space = {
                'param': hp.uniform('param', np.log(0.4), np.log(0.6))
#     "": 0.000001 + hp.uniform("gamma", 0.000001, 10)
    #             'param2': hp.loguniform('param2', np.log(0.001), np.log(0.01))
}

train.sample_plot(sample_space, 1000)

## Model performance evaluation - standard models

<a id = 'Model-performance-evaluation-standard-models'></a>

In [ ]:
top_models = train.top_bayes_optim_models(bayes_optim_summary=bayes_optim_summary, num_models=1)
top_models

In [ ]:
## standard model fit and predict
# select estimator and iteration
estimator = "LGBMRegressor"; model_iter = 417
# estimator = "XGBRegressor"; model_iter = 418
# estimator = "RandomForestRegressor"; model_iter = 382
# estimator = "GradientBoostingRegressor"; model_iter = 238
# estimator = "SVR"; model_iter = 259

# extract params and instantiate model
model = train.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary, estimator=estimator, model_iter=model_iter
)
model.fit(train.data.values, train.target.values)

X_train, X_valid, y_train, y_valid = train_test_split(train.data, train.target)
y_pred = model.predict(train.data.values)

In [ ]:

feature_selector_summary = train.regression_panel(
    model=model,
    X_train=train.data,
    y_train=train.target,
    X_valid=None,
    y_valid=None,
#     X_train=X_train,
#     y_train=y_train,
#     X_valid=X_valid,
#     y_valid=y_valid,
    n_folds=4,
    random_state=10,
#     feature_selector_summary=None
)
feature_selector_summary

In [ ]:
results = pd.DataFrame(data={'prediction' : model.predict(train.data.values),
                  'actual' : train.target.values
                  },
             index=train.target.index
            )
results['diff'] = results['prediction'] - results['actual']
results['diffAbs'] = abs(results['prediction'] - results['actual'])
results['diffPerc'] = ((results['prediction'] - results['actual']) / results['actual']) * 100

results.sort_values(['diffAbs'], ascending = False)[:10]

## Model explanability

<a id = 'Feature-importance'></a>

In [ ]:
# 
estimator = "XGBRegressor"; model_iter = 418

model = train.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary, estimator=estimator, model_iter=model_iter
)

model.fit(train.data.values, train.target.values)

##### Permutation importance

In [ ]:
# permutation importance - how much does performance decrease when shuffling a certain feature?
perm = PermutationImportance(model.model, random_state=1).fit(train.data, train.target)
eli5.show_weights(perm, feature_names=train.data.columns.tolist())

##### SHAP values - training data

###### Force plots - single observations

In [ ]:
# SHAP force plots for individual observations
for i in train.data.index[:5]:
    train.single_shap_viz_tree(obsIx=i, model=model, data=train.data, target=train.target, classification=False)

###### Force plots - multiple observations

In [ ]:
# SHAP force plot a set of data
visual = train.multi_shap_viz_tree(obs_ixs=train.data.index, model=model, data=train.data)
visual

###### Dependence plots

In [ ]:
# generate SHAP values for set of observations
obs_data, _, obs_shap_values = train.multi_shap_value_tree(
    obs_ixs=train.data.index, model=model, data=train.data
)

In [ ]:
# SHAP dependence plot grid
grid_features = ["OverallCond","LotFrontage","TotalSF","BsmtFinSF","LotConfig"]

train.shap_dependence_grid(
    obs_data=obs_data,
    obs_shap_values=obs_shap_values,
    grid_features=grid_features,
    all_features=train.data.columns,
    dot_size=35,
    alpha=0.5,
)

In [ ]:
# single SHAP dependence plot
p = PrettierPlot()
ax = p.make_canvas()

train.shap_dependence_plot(
    obs_data=obs_data,
    obs_shap_values=obs_shap_values,
    scatter_feature="TotalSF",
    color_feature="LotFrontage",
    feature_names=train.data.columns.tolist(),
    dot_size=50,
    alpha=0.5,
    ax=ax    
)

In [ ]:
# SHAP dependence plots for all feature relative to an interaction feature
feature_names = train.data.columns.tolist()
top_shap = np.argsort(-np.sum(np.abs(obs_shap_values), 0))

# generate force plot
for top_ix in top_shap:
    p = PrettierPlot()
    ax = p.make_canvas()
    
    train.shap_dependence_plot(
        obs_data=obs_data,
        obs_shap_values=obs_shap_values,
        scatter_feature=feature_names[top_ix],
        color_feature="Age",
        feature_names=feature_names,
        dot_size=35,
        alpha=0.5,
        ax=ax
    )

###### Summary plots

In [ ]:
# SHAP summary plot
feature_names = train.data.columns.tolist()
train.shap_summary_plot(
        obs_data=obs_data,
        obs_shap_values=obs_shap_values,
        feature_names=feature_names,
    )


##### SHAP values - talidation

###### Force plots - single observations

In [ ]:
# SHAP force plots for individual observations
for i in valid.data.index[:5]:
    valid.single_shap_viz_tree(obsIx=i, model=model, data=valid.data, classification=False)

###### Force plots - multiple observations

In [ ]:
# SHAP force plot a set of data
visual = valid.multi_shap_viz_tree(obs_ixs=valid.data.index, model=model, data=valid.data)
visual

###### Dependence plots

In [ ]:
# generate SHAP values for set of observations
obs_data, _, obs_shap_values = valid.multi_shap_value_tree(
    obs_ixs=valid.data.index, model=model, data=valid.data
)

In [ ]:
# SHAP dependence plot grid
grid_features = ["OverallCond","LotFrontage","TotalSF","BsmtFinSF","LotConfig"]

valid.shap_dependence_grid(
    obs_data=obs_data,
    obs_shap_values=obs_shap_values,
    grid_features=grid_features,
    all_features=valid.data.columns,
    dot_size=35,
    alpha=0.5,
)

In [ ]:
# single SHAP dependence plot
p = PrettierPlot()
ax = p.make_canvas()

valid.shap_dependence_plot(
    obs_data=obs_data,
    obs_shap_values=obs_shap_values,
    scatter_feature="TotalSF",
    color_feature="LotFrontage",
    feature_names=valid.data.columns.tolist(),
    dot_size=50,
    alpha=0.5,
    ax=ax    
)

In [ ]:
# SHAP dependence plots for all feature relative to an interaction feature
feature_names = valid.data.columns.tolist()
top_shap = np.argsort(-np.sum(np.abs(obs_shap_values), 0))

# generate force plot
for top_ix in top_shap:
    p = PrettierPlot()
    ax = p.make_canvas()
    
    valid.shap_dependence_plot(
        obs_data=obs_data,
        obs_shap_values=obs_shap_values,
        scatter_feature=feature_names[top_ix],
        color_feature="Age",
        feature_names=feature_names,
        dot_size=35,
        alpha=0.5,
        ax=ax
    )

###### Summary plots

In [ ]:
# SHAP summary plot
feature_names = valid.data.columns.tolist()
valid.shap_summary_plot(
        obs_data=obs_data,
        obs_shap_values=obs_shap_values,
        feature_names=feature_names,
    )


## Submission - standard models

<a id = 'Submission-standard-models'></a>

In [ ]:
## standard model fit and predict
# select estimator and iteration
estimator = "LGBMClassifier"; model_iter = 668
# estimator = "XGBClassifier"; model_iter = 380
# estimator = "RandomForestClassifier"; model_iter = 411
# estimator = "GradientBoostingClassifier"; model_iter = 590
# estimator = "SVC"; model_iter = 135

# extract params and instantiate model
model = train.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary, estimator=estimator, model_iter=model_iter
)
model.fit(train.data.values, train.target.values)

# fit model and make predictions
y_pred = model.predict(valid.data.values)

In [ ]:
# generate prediction submission file
submit = pd.DataFrame({"Id": df_test.Id, "SalePrice": np.expm1(y_pred)})
submit.to_csv("data/submission.csv", index=False)

# Stacking

<a id = 'Stacking'></a>

## Primary models

<a id = 'Primary-models'></a>

In [ ]:
# get out-of-fold predictions
oof_train, oof_valid, columns = train.model_stacker(
    models=top_models,
    bayes_optim_summary=bayes_optim_summary,
    X_train=train.data.values,
    y_train=train.target.values,
    X_valid=valid.data.values,
    n_folds=10,
    n_jobs=10,
)

In [ ]:
# view correlations of predictions
p = PrettierPlot()
ax = p.make_canvas()
p.corr_heatmap(
    df=pd.DataFrame(oof_train, columns=columns), annot=True, ax=ax, vmin=0
)

## Meta model

<a id = 'Meta-model'></a>

In [ ]:
# model/parameter space
all_space = {
    "KernelRidge": {
        "alpha": hp.uniform("alpha", 0.000001, 15),
        "kernel": hp.choice("kernel", ["linear", "polynomial", "rbf"]),
        "degree": hp.choice("degree", [2, 3]),
        "gamma": hp.uniform("gamma", 0.0, 10),
    },
    "LGBMRegressor": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "boosting_type": hp.choice("boosting_type", ["gbdt", "dart", "goss"])
        # ,'boosting_type': hp.choice('boosting_type'
        #                    ,[{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}
        #                    ,{'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)}
        #                    ,{'boosting_type': 'goss', 'subsample': 1.0}])
        ,
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_child_samples": hp.uniform("min_child_samples", 20, 500),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "num_leaves": hp.uniform("num_leaves", 8, 150),
        "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "subsample_for_bin": hp.uniform("subsample_for_bin", 20000, 400000),
    },
    "XGBRegressor": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "gamma": hp.uniform("gamma", 0.0, 10),
        "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_child_weight": hp.uniform("min_child_weight", 1, 20),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "subsample": hp.uniform("subsample", 0.5, 1),
    },
    "RandomForestRegressor": {
        "bootstrap": hp.choice("bootstrap", [True, False]),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "GradientBoostingRegressor": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "loss": hp.choice("loss", ["ls", "lad", "huber", "quantile"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "SVR": {
        "C": hp.uniform("C", 0.00001, 10),
        "kernel": hp.choice("kernel", ["linear", "poly", "rbf", "sigmoid"]),
        "degree": hp.choice("degree", [2, 3]),
        "gamma": hp.uniform("gamma", 0.0001, 10),
        "epsilon": hp.uniform("epsilon", 0.001, 5),
    },
}

In [ ]:
# execute bayesian optimization grid search
train.exec_bayes_optim_search(
    all_space=all_space,
    results_dir="{}_hyperopt_meta_{}.csv".format(rundate, analysis),
    X=oof_train,
    y=train.target,
    scoring="accuracy",
    n_folds=8,
    n_jobs=10,
    iters=1000,
    verbose=0,
)

In [ ]:
# read scores summary table
analysis = "housing"
rundate = "20190807"
bayes_optim_summary_meta = pd.read_csv("{}_hyperopt_meta_{}.csv".format(rundate, analysis))
bayes_optim_summary_meta[:5]

In [ ]:
# model loss plot
for estimator in np.unique(bayes_optim_summary_meta["estimator"]):
    train.model_loss_plot(bayes_optim_summary=bayes_optim_summary_meta, estimator=estimator)

In [ ]:
# estimator parameter plots
for estimator in np.unique(bayes_optim_summary_meta["estimator"]):
    train.modelParamPlot(
        bayes_optim_summary=bayes_optim_summary_meta,
        estimator=estimator,
        all_space=all_space,
        n_iter=100,
        chart_scale=15,
    )

## Model performance evaluation - stacked models

<a id = 'Model-performance-evaluation-stacked-models'></a>

In [ ]:
top_models = train.top_bayes_optim_models(
    bayes_optim_summary=bayes_optim_summary_meta, num_models=1
)
top_models

In [ ]:
# best second level learning model
estimator = "LGBMClassifier"; model_iter = 668
# estimator = "XGBClassifier"; model_iter = 380
# estimator = "RandomForestClassifier"; model_iter = 411
# estimator = "GradientBoostingClassifier"; model_iter = 590
# estimator = "SVC"; model_iter = 135

# extract params and instantiate model
model = train.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary_meta, estimator=estimator, model_iter=model_iter
)

# single model evaluation here

In [ ]:
# ,multi model evaluation here

## Submission - stacked models

<a id = 'Submission-stacked-models'></a>

In [ ]:
# best second level learning model
estimator = "LGBMClassifier"; model_iter = 668
# estimator = "XGBClassifier"; model_iter = 380
# estimator = "RandomForestClassifier"; model_iter = 411
# estimator = "GradientBoostingClassifier"; model_iter = 590
# estimator = "SVC"; model_iter = 135

# extract params and instantiate model
model = train.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary_meta, estimator=estimator, model_iter=model_iter
)

model.fit(oof_train, train.target.values)
y_pred = model.predict(oof_valid)
# print(sum(y_pred))

In [ ]:
# generate prediction submission file
submit = pd.DataFrame({"Id": df_test.Id, "SalePrice": np.expm1(y_pred)})
submit.to_csv("data/submission.csv", index=False)